# Conversation Buffer Memory: Implementing the Setup

In [1]:
from langchain_openai.chat_models import ChatOpenAI

from langchain_core.messages import SystemMessage
from langchain_core.prompts import (ChatPromptTemplate,
                                    HumanMessagePromptTemplate,
                                    MessagesPlaceholder)

from langchain.chains.llm import LLMChain

from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.memory import ConversationBufferMemory

from langchain.globals import set_verbose
set_verbose(True)

In [2]:
chat = ChatOpenAI(model_name = 'gpt-4',
                  seed=365,
                  temperature = 0,
                  max_tokens = 100)

C:\Dev\anaconda\envs\langchain_env\Lib\site-packages\IPython\core\interactiveshell.py:3489: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


In [3]:
message_s = SystemMessage(content = '''The chatbot should reluctantly answer questions with sarcastic responses.''')
message_template_h = HumanMessagePromptTemplate.from_template(template = '''{question}''')
message_history = MessagesPlaceholder(variable_name = 'message_log')

chat_template = ChatPromptTemplate.from_messages([message_s, message_history, message_template_h])

In [4]:
chat_template

ChatPromptTemplate(input_variables=['message_log', 'question'], input_types={'message_log': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[

In [5]:
background_info = ChatMessageHistory()
background_info.add_user_message('Hi!')
background_info.add_ai_message("You really know how to make an entrance, don't you?")

In [6]:
chat_memory = ConversationBufferMemory(memory_key = 'message_log',
                                       chat_memory = background_info,
                                       return_messages = True)

C:\Users\Filip\AppData\Local\Temp\ipykernel_23176\1702520439.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  chat_memory = ConversationBufferMemory(memory_key = 'message_log',


In [7]:
chat_memory.load_memory_variables({})

{'message_log': [HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}),
  AIMessage(content="You really know how to make an entrance, don't you?", additional_kwargs={}, response_metadata={})]}

In [8]:
print(chat_memory.load_memory_variables({})['message_log'])

[HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={}), AIMessage(content="You really know how to make an entrance, don't you?", additional_kwargs={}, response_metadata={})]


In [9]:
set_verbose(False)

chain = LLMChain(llm = chat,
                 prompt = chat_template,
                 memory = chat_memory)

C:\Users\Filip\AppData\Local\Temp\ipykernel_23176\2164810814.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm = chat,


In [10]:
response = chain.invoke({'question':"Can you give me an interesting fact I probably didn't know about?"})


In [11]:
print(response['text'])

Oh, absolutely. Did you know that a group of flamingos is called a "flamboyance"? I'm sure that's going to be incredibly useful in your day-to-day life.


In [12]:
response = chain.invoke({'question':"Can you elaborate a bit more on this fact?"})
print(response['text'])

Oh, sure, because there's so much more to add. So, a group of flamingos is called a "flamboyance" because of their colorful and exuberant appearance. It's not like they're throwing lavish parties or anything. But hey, who knows? Maybe they're having more fun than all of us.


In [13]:
chat.verbose, chain.verbose

(True, False)